In [11]:
import numpy as np
import polars as pl

import altair as alt
from altair import datum
import vegafusion as vf
vf.enable(row_limit=100000)

vegafusion.enable(mimetype='html', row_limit=100000, embed_options=None)

In [12]:
# df_gcc = pl.scan_parquet('../../m5out/trace_gcc_lto.parquet', low_memory=True, cache=False)
df_m88ksim = pl.scan_parquet('../../m5out/trace_m88ksim_lto.parquet', low_memory=True, cache=False)
df_go = pl.scan_parquet('../../m5out/trace_go_lto.parquet', low_memory=True, cache=False)
df_gcc = pl.scan_parquet('../../m5out/trace_gcc_lto.parquet', low_memory=True, cache=False)

df_t1 = pl.scan_parquet('../../m5out/trace1.parquet', low_memory=True, cache=False)
df_t2 = pl.scan_parquet('../../m5out/trace2.parquet', low_memory=True, cache=False)


In [3]:
df_t1.select(pl.count('tick').alias('num_rows')).collect(streaming=True)

num_rows
u32
703722


In [4]:
avg_metrics_df = df_t1.groupby('inst_rel_addr').agg([
    pl.count(), 
    pl.col('taken').cast(pl.UInt64).mean(),
    pl.col('mispredicted').cast(pl.UInt64).mean()
]).collect(streaming=True)

In [5]:
taken_plot = alt.Chart(avg_metrics_df).mark_circle().encode(
    x = 'taken:Q',
    y = alt.Y(
        'count:Q',
        scale=alt.Scale(type="log")
    ),
    tooltip=['inst_rel_addr:O', 'count:Q', 'total_taken:Q']
).transform_calculate(
    total_taken = 'floor(datum.taken * datum.count)'
).interactive()

logdata = pl.DataFrame({'x': np.linspace(0.0001,1,100,dtype=float), 'y': 1 / np.linspace(0.0001,1,100,dtype=float) })
logplot = alt.Chart(logdata).mark_line(color="#FFAA00").encode(
    x = 'x:Q',
    y = alt.Y(
        'y:Q',
        scale=alt.Scale(type="log")
    )
).interactive()

mispred_plot = alt.Chart(avg_metrics_df).mark_circle().encode(
    x = 'mispredicted:Q',
    y = alt.Y(
        'count:Q',
        scale=alt.Scale(type="log")
    ),
    tooltip='inst_rel_addr:O'
).interactive()

# ((logplot+taken_plot) | mispred_plot)
((taken_plot) | mispred_plot)


alt.HConcatChart(...)

$$
r_{\text{taken}} = \frac{\text{count}_\text{taken}}{\text{count}_\text{total}}, \qquad \text{count}_\text{taken}\in\mathbb{N}, \text{count}_\text{taken} \leq \text{count}_\text{total}
$$

$$
\text{count}_\text{total} = \frac{\text{count}_\text{taken}}{r_{\text{taken}}}
$$

In [17]:
bname = 'clone_candidate1+12c'

ddf = df_t1.filter(pl.col('inst_rel_addr') == bname) \
    .groupby(pl.col('ras').arr.head(1)) \
    .agg([
        pl.count(),
        pl.col('taken').cast(pl.UInt64).mean(),
        pl.col('mispredicted').cast(pl.UInt64).mean()
    ]).filter(pl.col('count') > 10).collect()
ddf.head()

ras,count,taken,mispredicted
list[u64],u32,f64,f64
[4198952],509,0.375246,0.328094


In [45]:
bname = 'gen_rtx+54'

ddf = df_gcc.filter(pl.col('inst_rel_addr') == bname) \
    .groupby(pl.col('ras').arr.head(1)) \
    .agg([
        pl.count(),
        pl.col('taken').cast(pl.UInt64).mean(),
        pl.col('mispredicted').cast(pl.UInt64).mean()
    ]).filter((pl.col('count') > 10) &  (pl.col('mispredicted') >= 0.5)) \
    .sort('mispredicted', descending=True) \
    .collect()
ddf.head(100)

ras,count,taken,mispredicted
list[u64],u32,f64,f64
[4483988],35,1.0,1.0
[4208156],17,1.0,1.0
[5158688],18,1.0,0.888889
[5215824],100,0.0,0.82
[4861132],51,1.0,0.803922
[4278772],26,0.0,0.769231
[4751464],38,0.0,0.736842
[4723680],26,0.0,0.730769
[4278300],21,0.0,0.714286


In [8]:
bns = df_gcc.groupby(pl.col('inst_addr')) \
        .agg(pl.col('mispredicted').cast(pl.UInt64).mean()) \
        .filter(pl.col('mispredicted') > 0.4) \
        .select(pl.col('inst_addr')).collect(streaming=True)['inst_addr']

ddf2 = df_gcc.filter(pl.col('inst_addr').is_in(bns)) \
    .groupby([pl.col('inst_rel_addr'), pl.col('inst_addr'), pl.col('ras').arr.first()]) \
    .agg([
        pl.count(), 
        pl.col('taken').cast(pl.UInt64).mean()
    ]) \
    .filter(pl.col('count') > 100) \
    .groupby(pl.col('inst_rel_addr'), pl.col('inst_addr')) \
    .agg([pl.col('taken').std(), pl.col('count').sum()]) \
    .filter(pl.col('taken') > 0.1) \
    .select(pl.col('inst_rel_addr'), pl.col('inst_addr'), pl.col('taken').sort(descending=True), pl.col('count').sort()) \
    .collect(streaming=True)

In [21]:
ddf2

inst_rel_addr,inst_addr,taken,count
str,u64,f64,u32
"""expand_goto_in…",5172440,0.707107,209
"""build_binary_o…",5358376,0.707107,280
"""cse_end_of_bas…",4763344,0.707107,287
"""num_sign_bit_c…",4618556,0.707107,309
"""subst.lto_priv…",4655800,0.707107,321
"""fill_simple_de…",4444920,0.707107,321
"""qsort+a8""",4288784,0.707107,334
"""pushdecl+348""",5392224,0.700793,349
"""simplify_plus_…",4818728,0.682489,361


In [9]:
ddf2.select(pl.count())

count
u32
61


In [38]:
df_go.filter(pl.col('virtual')).select(pl.count()).collect(streaming=True)

count
u32
24890


In [48]:
df_m88ksim.filter(pl.col('virtual')).head(50).collect(streaming=True)

tick,disassembly,inst_addr,inst_rel_addr,pred_addr,pred_rel_addr,jump_addr,jump_rel_addr,pred_taken,mispredicted,ras,regs,virtual,return,call,taken,ras_rel
u64,str,u64,str,u64,str,u64,str,bool,bool,list[u64],struct[15],bool,bool,bool,bool,list[str]
2228000,""" ret """,4413308,"""_dl_aux_init+1…",4279372,"""__libc_start_m…",4279372,"""__libc_start_m…",true,false,"[4279368, 4197244]","{549755748188,0,4294967295,0,0,1,4999192,4997120,4096,4890624,5001216,5001216,1,4915200,5001216}",true,true,false,true,"[""__libc_start_main+80"", ""_start+48""]"
2350000,""" ret """,4416340,"""__libc_init_se…",4279404,"""__libc_start_m…",4279404,"""__libc_start_m…",true,false,"[4279400, 4197244]","{4653056,5001216,4194368,0,0,1,4999192,4997120,4096,4890624,5001216,5001216,1,4915200,5001216}",true,true,false,true,"[""__libc_start_main+a0"", ""_start+48""]"
2681000,""" ret """,4411836,"""__tunables_ini…",4279416,"""__libc_start_m…",4279416,"""__libc_start_m…",true,false,"[4279412, 4197244]","{0,5001216,4194368,0,0,1,4999192,4997120,4096,4890624,5001216,5001216,1,4915200,5001216}",true,true,false,true,"[""__libc_start_main+ac"", ""_start+48""]"
2976000,""" ret """,4412772,"""__tunable_get_…",4279432,"""__libc_start_m…",4279432,"""__libc_start_m…",true,false,"[4279428, 4197244]","{3,549755747608,0,4893240,4893072,3,4999192,4997120,4096,4890624,5001216,5001216,1,4915200,5001216}",true,true,false,true,"[""__libc_start_main+bc"", ""_start+48""]"
3349000,""" blr x2""",4279600,"""__libc_start_m…",4279604,"""__libc_start_m…",4382800,"""__libc_memmove…",false,true,[4197244],"{1032,4,4999216,0,4893072,3,4999192,4997120,4096,4890624,5001216,5001216,1,4915200,5001216}",true,false,true,true,"[""_start+48""]"
3770000,""" ret """,4383064,"""__libc_memmove…",4279604,"""__libc_start_m…",4279604,"""__libc_start_m…",true,false,"[4279600, 4197244]","{0,0,4382800,0,4893072,3,4999192,4997120,4096,4890624,5001216,5001216,1,4915200,5001216}",true,true,false,true,"[""__libc_start_main+168"", ""_start+48""]"
3883000,""" blr x2""",4279600,"""__libc_start_m…",4279604,"""__libc_start_m…",4383920,"""__libc_memcpy+…",false,true,[4197244],"{0,0,4382800,0,4893072,3,4999192,4997120,4096,4890624,5001216,5001216,1,4915200,5001216}",true,false,true,true,"[""_start+48""]"
4470000,""" ret """,4384224,"""__libc_memcpy+…",4279604,"""__libc_start_m…",4279604,"""__libc_start_m…",true,false,"[4279600, 4197244]","{0,0,4383920,0,4893072,3,4999192,4997120,4096,4890624,5001216,5001216,1,4915200,5001216}",true,true,false,true,"[""__libc_start_main+168"", ""_start+48""]"
4480000,""" blr x2""",4279600,"""__libc_start_m…",4279604,"""__libc_start_m…",4382336,"""__memchr+0""",false,true,[4197244],"{0,0,4383920,0,4893072,3,4999192,4997120,4096,4890624,5001216,5001216,1,4915200,5001216}",true,false,true,true,"[""_start+48""]"


In [20]:
d = df_gcc.select([pl.col('inst_addr'), pl.col('ras').arr.first()]) \
      .groupby(pl.col('inst_addr')) \
      .agg(pl.col('ras').n_unique()).sort('ras', descending=True).collect(streaming=True)

In [22]:
d.head(100)

inst_addr,ras
u64,u32
4939380,600
4939316,600
4939396,600
4939516,515
4939908,463
4939744,459
4939800,459
4939836,455
4939832,455


In [40]:
df_gcc.filter(pl.col('inst_addr') == 4939316).first().select(pl.col('inst_rel_addr')).collect()

inst_rel_addr
str
"""gen_rtx+54"""


In [32]:
pl.Config.set_fmt_str_lengths(100)
df_gcc.filter(pl.col('inst_addr') == 4939396).select(pl.col('ras_rel').arr.first().unique()).collect(streaming=True)

ras_rel
str
"""compile_file.lto_priv.0+1ce8"""
"""init_expmed+1c8"""
"""make_extraction.lto_priv.0+850"""
"""gen_seq_special+10c"""
"""get_condition.part.0+408"""
"""init_expr_once+50"""
"""copy_loop_body.lto_priv.0+cb4"""
"""copy_rtx_and_substitute+230"""
"""gen_seq_special+a4"""
